In [1]:
import scipy.io
import torch
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Load MATLAB data
mat_data = scipy.io.loadmat('data/condsForSimJ2moMuscles.mat')# Load MATLAB data

In [3]:
# Access the 'condsForSim' array
condsForSim = mat_data['condsForSim']

In [4]:
import torch

# Initialize a list to hold the tensors
goEnvelopes = []

# Loop through each condition
for i in range(condsForSim.shape[0]):  # 27 rows
    for j in range(condsForSim.shape[1]):  # 8 columns
        condition = condsForSim[i, j]

        # Extract goEnvelope and convert to tensor
        goEnvelope = condition['goEnvelope']
        goEnvelope_tensor = torch.tensor(goEnvelope, dtype=torch.float32).view(-1, 1)  # Reshape to [296, 1]
        goEnvelopes.append(goEnvelope_tensor)

# Concatenate all sequences along a new dimension
goEnvelope_all = torch.stack(goEnvelopes)  # Shape: [216, 296, 1]

print(goEnvelope_all.shape)


torch.Size([216, 296, 1])


In [5]:
import torch

# Initialize a list to hold the tensors
plans = []

# Loop through each condition
for i in range(condsForSim.shape[0]):  # 27 rows
    for j in range(condsForSim.shape[1]):  # 8 columns
        condition = condsForSim[i, j]

        # Extract plan data
        plan = condition['plan']  # Accessing the data; ensure to use [0, 0] if it's nested like goEnvelope
        plan_tensor = torch.tensor(plan, dtype=torch.float32)  # Keep its original shape [296, 15]
        plans.append(plan_tensor)

# Stack all plan sequences
plan_all = torch.stack(plans)  # Shape: [216, 296, 15]

print(plan_all.shape)


torch.Size([216, 296, 15])


In [6]:
import torch

# Initialize a list to hold the tensors
muscles = []

# Loop through each condition
for i in range(condsForSim.shape[0]):  # 27 rows
    for j in range(condsForSim.shape[1]):  # 8 columns
        condition = condsForSim[i, j]

        # Extract plan data
        muscle = condition['muscle']  # Accessing the data; ensure to use [0, 0] if it's nested like goEnvelope
        muscle_tensor = torch.tensor(muscle, dtype=torch.float32)  # Keep its original shape [296, 15]
        muscles.append(muscle_tensor)

# Stack all plan sequences
muscle_all = torch.stack(muscles)  # Shape: [216, 296, 15]

print(muscle_all.shape)


torch.Size([216, 296, 8])


In [7]:
from torch.utils.data import TensorDataset, DataLoader

# Assuming plan_tensor and goEnvelope_tensor are inputs and muscle_tensor is the output
dataset = TensorDataset(plan_tensor, goEnvelope_tensor, muscle_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [11]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through RNN layers
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

# Define the dimensions for your model based on the input and output data sizes
input_size = 15  # The feature dimension of the input data
hidden_size = 64  # You can choose an appropriate hidden size
num_layers = 2    # You can choose the number of RNN layers
output_size = 8   # The feature dimension of the output data

In [12]:
# Instantiate the model with the defined dimensions
model = RNNModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [13]:
num_epochs = 10  # Set the number of epochs

for epoch in range(num_epochs):
    # Loop over the data_loader
    for inputs, targets in data_loader:
        optimizer.zero_grad()  # Reset the gradients to zero in the optimizer
        outputs = model(inputs)  # Pass the batch of inputs through the model
        loss = criterion(outputs, targets)  # Compute the loss
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model's parameters

    # Validation step
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    total_val_accuracy = 0  # If accuracy is a relevant metric

    with torch.no_grad():  # Disable gradient computation
        for val_inputs, val_targets in val_loader:
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_targets)
            total_val_loss += val_loss.item()

            # Calculate accuracy if relevant
            _, predicted = torch.max(val_outputs.data, 1)
            correct = (predicted == val_targets).sum().item()
            total_val_accuracy += correct / val_targets.size(0)

    # Calculate average validation loss and accuracy
    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_accuracy = total_val_accuracy / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.4f}')

    # Set the model back to training mode
    model.train()


NameError: name 'data_loader' is not defined